In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train', './input/test']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import numpy as np
# import pandas as pd
# FIRST-AUTHOR: remove plotting, ML code
# from matplotlib import pyplot as plt
# import seaborn as sns
# from sklearn.preprocessing import PowerTransformer

In [3]:
#データの読み込み

train = pd.read_csv('./input/train.scaled.csv') #訓練データ
test_x = pd.read_csv('./input/test.scaled.csv') #テストデータ

#学習データの変数を確認
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
#目的変数である家の価格の要約統計量を表示する
train["SalePrice"].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [5]:
#50パーセントタイルが16万なのに対して平均が18万となっている、かつMAXが75万と大きいので正規分布にはなってない気がします。ヒストグラムを見てみます。

In [6]:
#目的変数である家の価格のヒストグラムを表示する
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# sns.distplot(train['SalePrice'])
_ = train['SalePrice']

In [7]:
#やはり正規分布になっていませんね。目的変数の分布は正規分布になっていないとモデルの予測精度に影響が出てしまうので、これはあとで対数変換する必要がありそうです。

In [8]:
#歪度と尖度を計算
print("歪度: %f" % train['SalePrice'].skew())
print("尖度: %f" % train['SalePrice'].kurt())

歪度: 1.882876
尖度: 6.536282


In [9]:
#歪度が正の値なので、右裾が長い（≒左に偏っている）分布であること、さらに尖度が正の値のため正規分布よりもだいぶ尖った（平均付近にデータが集中している）分布であることがわかります。

In [10]:
#ここで与えられた特徴量をみると

#物件の大きさ（広さ）
#築年数
#家の材質と完成度
#あたりがぱっとみで物件の価格に影響しそうなので、目的変数との関係を見てみます。

In [11]:
#物件の広さを合計した変数を作成
train["TotalSF"] = train["1stFlrSF"] + train["2ndFlrSF"] + train["TotalBsmtSF"]
test_x["TotalSF"] = test_x["1stFlrSF"] + test_x["2ndFlrSF"] + test_x["TotalBsmtSF"]

#物件の広さと物件価格の散布図を作成
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.scatter(train["TotalSF"],train["SalePrice"])
# plt.xlabel("TotalSF")
# plt.ylabel("SalePrice")
_ = train["TotalSF"]
_ = train["SalePrice"]

In [12]:
#若干外れ値がありますが、相関しているように見えます。やはり物件が大きくなるほど物件価格も高くなることがわかります。

In [13]:
#外れ値を除外する
train = train.drop(train[(train['TotalSF']>7500) & (train['SalePrice']<300000)].index)

#物件の広さと物件価格の散布図を作成
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.scatter(train["TotalSF"],train["SalePrice"])
# plt.xlabel("TotalSF")
# plt.ylabel("SalePrice")
_ = train["TotalSF"]
_ = train["SalePrice"]

In [14]:
#築年数と物件価格の散布図を作成
#plt.scatter(train["YearBuilt"],train["SalePrice"],color = "#e41a1c")
#plt.xlabel("YearBuilt")
#plt.ylabel("SalePrice")

data = pd.concat([train["YearBuilt"],train["SalePrice"]],axis=1)

# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.xticks(rotation='90')
# sns.boxplot(x="YearBuilt",y="SalePrice",data=data)

In [15]:
#微妙ですが築年数が新しいほど物件価格が高くなる傾向はありそうです。
#こちらも外れ値があるので除外します。

In [16]:
#外れ値を除外する
train = train.drop(train[(train['YearBuilt']<2000) & (train['SalePrice']>600000)].index)

#グラフを描画する
data = pd.concat([train["YearBuilt"],train["SalePrice"]],axis=1)

# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.xticks(rotation='90')
# sns.boxplot(x="YearBuilt",y="SalePrice",data=data)

In [17]:
#さらに以下のような特徴量があります。これを家の材質と完成度と訳すのがよいのかわかりませんが価格には影響与えそうなので可視化してみます。

In [18]:
#家の材質・完成度と物件価格の散布図を作成
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.scatter(train["OverallQual"],train["SalePrice"])
# plt.xlabel("OverallQual")
# plt.ylabel("SalePrice")
_ = train["OverallQual"]
_ = train["SalePrice"]

In [19]:
#クオリティが上がるほど価格が上がっているように見えます。

In [20]:
#外れ値を除外する
train = train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index)
train = train.drop(train[(train['OverallQual']<10) & (train['SalePrice']>500000)].index)

#グラフを描画する
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# plt.scatter(train["OverallQual"],train["SalePrice"])
# plt.xlabel("OverallQual")
# plt.ylabel("SalePrice")
_ = train["OverallQual"]
_ = train["SalePrice"]

In [21]:
#前処理をいっぺんに行うために、学習データとテストデータをマージします。 さらに学習データに関しては目的変数である「SalesPrice」が含まれているのでこちらは切り出しておきます。

In [22]:
#学習データを目的変数とそれ以外に分ける
train_x = train.drop("SalePrice",axis=1)
train_y = train["SalePrice"]

#学習データとテストデータを統合
all_data = pd.concat([train_x,test_x],axis=0,sort=True)

#IDのカラムは不必要なので別の変数に格納
train_ID = train['Id']
test_ID = test_x['Id']

all_data.drop("Id", axis = 1, inplace = True)

#それぞれのデータのサイズを確認
print("train_x: "+str(train_x.shape))
print("train_y: "+str(train_y.shape))
print("test_x: "+str(test_x.shape))
print("all_data: "+str(all_data.shape))

train_x: (1449, 81)
train_y: (1449,)
test_x: (1459, 81)
all_data: (2908, 80)


In [23]:
#続いて欠損値を処理していきたいと思います。まずはどのくらい欠損値があるのかを確認します。

In [24]:
#データの欠損値を確認する
all_data_na = all_data.isnull().sum()[all_data.isnull().sum()>0].sort_values(ascending=False)
all_data_na

PoolQC          2900
MiscFeature     2803
Alley           2710
Fence           2338
FireplaceQu     1420
LotFrontage      485
GarageCond       159
GarageQual       159
GarageYrBlt      159
GarageFinish     159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
Utilities          2
Functional         2
BsmtHalfBath       2
BsmtFullBath       2
GarageCars         1
Exterior2nd        1
Exterior1st        1
KitchenQual        1
Electrical         1
BsmtUnfSF          1
BsmtFinSF2         1
BsmtFinSF1         1
SaleType           1
TotalBsmtSF        1
TotalSF            1
GarageArea         1
dtype: int64

In [25]:
#めちゃくちゃありますね、、、グラフ化してみます。

In [26]:
#欠損値の数をグラフ化
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20,10))
# plt.xticks(rotation='90')
# sns.barplot(x=all_data_na.index, y=all_data_na)
_ = all_data_na.index

In [27]:
#欠損値については一括で削除するか、平均値で置換してしまいたくなってしまいますが、
#きちんと欠損値が多い変数が何を表すものか見てみます。

#PoolQC: Pool quality
#備え付けられているプールの質を表す。プールがない場合にはNAとなる。
#MiscFeature: Miscellaneous feature not covered in other categories
#その他の備え付けられている設備を表す。エレベータやテニスコートなど。特にない場合はNAとなる。
#Alley: Type of alley access to property
#物件にアクセスするための道の種類（砂利なのか舗装されているのか）を表す。該当しない場合はNAとなる
#Fence: Fence quality
#フェンスの質を表す。フェンスがない場合はNAとなる。
#FireplaceQu: Fireplace quality
#暖炉の品質を表す。暖炉がない場合はNAとなる。
#LotFrontage: Linear feet of street connected to property
#物件に隣接した道路の長さ。
#欠損値が多い変数に関しては、データが欠損しているというわけではなく、欠損＝そもそもその設備がないことを表しているようですね。
#次に欠損値がある変数のデータ型を確認します。

In [28]:
# 欠損値があるカラムをリスト化
na_col_list = all_data.isnull().sum()[all_data.isnull().sum()>0].index.tolist()

#欠損があるカラムのデータ型を確認
all_data[na_col_list].dtypes.sort_values()

GarageArea      float64
TotalBsmtSF     float64
TotalSF         float64
BsmtFinSF1      float64
BsmtFinSF2      float64
GarageYrBlt     float64
BsmtFullBath    float64
BsmtHalfBath    float64
GarageCars      float64
BsmtUnfSF       float64
MasVnrArea      float64
LotFrontage     float64
GarageType       object
KitchenQual      object
MasVnrType       object
MiscFeature      object
PoolQC           object
SaleType         object
MSZoning         object
GarageQual       object
Alley            object
GarageCond       object
Functional       object
FireplaceQu      object
Fence            object
Exterior2nd      object
Exterior1st      object
Electrical       object
BsmtQual         object
BsmtFinType2     object
BsmtFinType1     object
BsmtExposure     object
BsmtCond         object
GarageFinish     object
Utilities        object
dtype: object

In [29]:
#floatとobjectですね。float型の場合は0、objectの場合は”None”で置換することにしましょう。
#ただし、隣接した道路の長さ（LotFrontage）に関しては同じ地区と他の物件と同じと思われるので、同じ地区の中央値をとることにします。

In [30]:
#隣接した道路の長さ（LotFrontage）の欠損値の補完
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

#欠損値が存在するかつfloat型のリストを作成
float_list = all_data[na_col_list].dtypes[all_data[na_col_list].dtypes == "float64"].index.tolist()

#欠損値が存在するかつobject型のリストを作成
obj_list = all_data[na_col_list].dtypes[all_data[na_col_list].dtypes == "object"].index.tolist()

#float型の場合は欠損値を0で置換
all_data[float_list] = all_data[float_list].fillna(0)

#object型の場合は欠損値を"None"で置換
all_data[obj_list] = all_data[obj_list].fillna("None")

#欠損値が全て置換できているか確認
all_data.isnull().sum()[all_data.isnull().sum() > 0]

Series([], dtype: int64)

In [31]:
#さきほどはカテゴリ変数を処理しましたが、続いては数値変数です。
#数値変数は基本はそのままモデルの入力に使えますが、データ上は数値であっても値の大きさや順番に意味のないものはカテゴリ変数として扱うべきです。
#もう一度データの定義を見てみると、以下の変数は数値変数ではなくカテゴリ変数として扱ったほうが良さそうです。

#MSSubClass: Identifies the type of dwelling involved in the sale
#住宅の種類を表す。数値はどの種類に当てはまるかを表すだけで大きさや順序に意味はない。
#YrSold: Year Sold (YYYY)
#販売年
#MoSold: Month Sold (MM)
#販売月

In [32]:
# カテゴリ変数に変換する
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [33]:
#探索的データ分析のフェーズで目的変数であるSalesPriceが正規分布になっていないことがわかりました。
#こちらを正規分布に変換します。

In [34]:
#目的変数の対数log(x+1)をとる
train_y = np.log1p(train_y)

#分布を可視化
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20, 10))
# sns.distplot(train_y)

In [35]:
#同様に説明変数に関しても正規分布にしたがっていないものは対数変換していきます。
#各説明変数に対して歪度を計算し、0.5よりも大きい場合は対数変換することにします。

In [36]:
#数値の説明変数のリストを作成
num_feats = all_data.dtypes[all_data.dtypes != "object" ].index

#各説明変数の歪度を計算
skewed_feats = all_data[num_feats].apply(lambda x: x.skew()).sort_values(ascending = False)

#グラフ化
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20,10))
# plt.xticks(rotation='90')
# sns.barplot(x=skewed_feats.index, y=skewed_feats)
_ = skewed_feats.index

In [37]:
#ちなみにBox-Cox変換は0以下の値をとる変数には使用できないため、各変数の最小値を見てみます。

In [38]:
#歪度の絶対値が0.5より大きい変数だけに絞る
skewed_feats_over = skewed_feats[abs(skewed_feats) > 0.5].index

#各変数の最小値を表示
for i in skewed_feats_over:
    print(min(all_data[i]))

0
0
1300
0
0
0
0.0
0
0
0.0
0.0
0
0
334
21.0
0.0
334
0.0
0.0
0
2
0
0
0.0
0.0
1
1872
0.0


In [39]:
#負の値はないですが、0が含まれる変数がありますね、、、
#今回はBox-Cox変換ではなく、０以下の値を持つ変数にも適用可能なYeo-Johnson変換を使いたいと思います。

In [40]:
#Yeo-Johnson変換
# FIRST-AUTHOR: remove ML code
# pt = PowerTransformer()
# pt.fit(all_data[skewed_feats_over])

# #変換後のデータで各列を置換
# all_data[skewed_feats_over] = pt.transform(all_data[skewed_feats_over])
_ = all_data[skewed_feats_over]
all_data[skewed_feats_over] = all_data[skewed_feats_over]

#各説明変数の歪度を計算
skewed_feats_fixed = all_data[skewed_feats_over].apply(lambda x: x.skew()).sort_values(ascending = False)

#グラフ化
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(20,10))
# plt.xticks(rotation='90')
# sns.barplot(x=skewed_feats_fixed.index, y=skewed_feats_fixed)
_ = skewed_feats_fixed.index

In [41]:
#同じくSalesPriceを予測する「Zillow Prize: Zillow’s Home Value Prediction」で、物件の面積を部屋数で割った１部屋当たりの面積という特徴量を追加したことで精度が上がった、ということがあったそうなので今回も追加してみます。

In [42]:
#特徴量に1部屋あたりの面積を追加
all_data["FeetPerRoom"] =  all_data["TotalSF"]/all_data["TotRmsAbvGrd"]

#その他有効そうなものを追加する

#建築した年とリフォームした年の合計
all_data['YearBuiltAndRemod']=all_data['YearBuilt']+all_data['YearRemodAdd']

#バスルームの合計面積
all_data['Total_Bathrooms'] = (all_data['FullBath'] + (0.5 * all_data['HalfBath']) +
                               all_data['BsmtFullBath'] + (0.5 * all_data['BsmtHalfBath']))

#縁側の合計面積
all_data['Total_porch_sf'] = (all_data['OpenPorchSF'] + all_data['3SsnPorch'] +
                              all_data['EnclosedPorch'] + all_data['ScreenPorch'] +
                              all_data['WoodDeckSF'])

#プールの有無
all_data['haspool'] = all_data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)

#2階の有無
all_data['has2ndfloor'] = all_data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)

#ガレージの有無
all_data['hasgarage'] = all_data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)

#地下室の有無
all_data['hasbsmt'] = all_data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

#暖炉の有無
all_data['hasfireplace'] = all_data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [43]:
#各カラムのデータ型を確認
all_data.dtypes.value_counts()

object     46
int64      29
float64    14
dtype: int64

In [44]:
#カテゴリ変数は46つあります。

#学習データには存在せず、テストデータのみに存在するカテゴリ変数が存在するとモデルがそのカテゴリを学習できず、予測値がおかしくなる可能性があります。
#テストデータのみに存在するカテゴリ変数が存在しないかを確認しましょう。

In [45]:
#カテゴリ変数となっているカラムを取り出す
cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()

#学習データにおけるカテゴリ変数のデータ数を確認
train_x[cal_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1449 entries, 0 to 1459
Data columns (total 46 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Alley          91 non-null     object
 1   BldgType       1449 non-null   object
 2   BsmtCond       1412 non-null   object
 3   BsmtExposure   1411 non-null   object
 4   BsmtFinType1   1412 non-null   object
 5   BsmtFinType2   1411 non-null   object
 6   BsmtQual       1412 non-null   object
 7   CentralAir     1449 non-null   object
 8   Condition1     1449 non-null   object
 9   Condition2     1449 non-null   object
 10  Electrical     1448 non-null   object
 11  ExterCond      1449 non-null   object
 12  ExterQual      1449 non-null   object
 13  Exterior1st    1449 non-null   object
 14  Exterior2nd    1449 non-null   object
 15  Fence          280 non-null    object
 16  FireplaceQu    759 non-null    object
 17  Foundation     1449 non-null   object
 18  Functional     1449 non-null

In [46]:
#学習データの中に数が0となっているカテゴリ変数はないようですのでそのまま進めていきます。

In [47]:
#カテゴリ変数のエンコーディング方法はさまざまありますのが、今回はone-hot-encodingによってハンドリングします。
#one-hot-encodingでは、各カテゴリ変数を（0,1）の二値変数をそれぞれ作成します。これらの二値変数は「ダミー変数」と呼ばれます。

#pandasのget_dummies関数でone-hot-encodingを行います。

In [48]:
#カテゴリ変数をget_dummiesによるone-hot-encodingを行う
all_data = pd.get_dummies(all_data,columns=cal_list)

#サイズを確認
all_data.shape

(2908, 349)

In [49]:
#前処理とエンコーディングが終わったのでデータを学習データとテストデータに分割しておきます。

In [50]:
#学習データとテストデータに再分割
train_x = all_data.iloc[:train_x.shape[0],:].reset_index(drop=True)
test_x = all_data.iloc[train_x.shape[0]:,:].reset_index(drop=True)

#サイズを確認
print("train_x: "+str(train_x.shape))
print("test_x: "+str(test_x.shape))

train_x: (1449, 349)
test_x: (1459, 349)


In [51]:
# FIRST-AUTHOR: remove ML code
# from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# from sklearn.metrics import mean_squared_error
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV
# import lightgbm as lgb

In [52]:
#モデルの評価用に学習データを学習に使用するデータと評価用データ（バリデーションデータ）に分割します。

In [53]:
# データの分割
# FIRST-AUTHOR: remove ML code
# train_x, valid_x, train_y, valid_y = train_test_split(
#         train_x,
#         train_y,
#         test_size=0.3,
#         random_state=0)

In [54]:
#最近、kaggleで人気だと言われているGBDT（勾配ブースティング木）でモデルを作成します。GBDTはランダムフォレストと勾配ブースティング（Gradient Boosting）を組み合わせたアンサンブルです。GBDTには以下の特徴があります。

#特徴量は数値である必要がある
#欠損値の補完が不要

In [55]:
#特徴量と目的変数をxgboostのデータ構造に変換する
# FIRST-AUTHOR: remove ML code
# dtrain = xgb.DMatrix(train_x, label=train_y)
# dvalid = xgb.DMatrix(valid_x,label=valid_y)

# #パラメータを指定してGBDT
# num_round = 5000
# evallist = [(dvalid, 'eval'), (dtrain, 'train')]

# evals_result = {}

# #パラメータ
# param = {
#             'max_depth': 3,
#             'eta': 0.01,
#             'objective': 'reg:squarederror',
# }

# #学習の実行
# bst = xgb.train(
#                         param, dtrain,
#                         num_round,
#                         evallist,
#                         evals_result=evals_result,
#                         # 一定ラウンド回しても改善が見込めない場合は学習を打ち切る
#                         early_stopping_rounds=1000
# )

In [56]:
#学習曲線を可視化する
# FIRST-AUTHOR: remove plotting, ML code
# plt.figure(figsize=(20, 10))
# train_metric = evals_result['train']['rmse']
# plt.plot(train_metric, label='train rmse')
# eval_metric = evals_result['eval']['rmse']
# plt.plot(eval_metric, label='eval rmse')
# plt.grid()
# plt.legend()
# plt.xlabel('rounds')
# plt.ylabel('rmse')
# plt.ylim(0, 0.3)
# plt.show()

In [57]:
#特徴量ごとの重要度を可視化する
# FIRST-AUTHOR: remove plotting
# ax = xgb.plot_importance(bst)
# fig = ax.figure
# fig.set_size_inches(10, 30)

In [58]:
# FIRST-AUTHOR: remove ML code
# dtest = xgb.DMatrix(test_x)
my_submission = pd.DataFrame()
my_submission["Id"] = test_ID
# FIRST-AUTHOR: remove ML code
# my_submission["SalePrice"] = np.exp(bst.predict(dtest))
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)